In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier

df = pd.read_excel('C:/Users/Enzo/Downloads/Trabalho/ArquivoFonte.xlsx')

df = df.sort_values(by='Dia')

df.head()


In [ ]:
# converter em valores em coluna categórica
encoders = {col: LabelEncoder() for col in ["Disciplina", "Professor", "Semestre", "Dia da Semana"]}

# Aplicar encoding nas colunas categóricas
for col, encoder in encoders.items():
    df[col] = encoder.fit_transform(df[col])
    
    
X = df[["Semestre", "Disciplina", "Professor"]]
y = df["Dia da Semana"]

df.head(30)

In [ ]:
# Dividir dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Treinar o modelo RandomForest
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# Avaliar o modelo
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Acurácia do modelo: {accuracy:.2f}")

In [ ]:
# Prever o dia da semana para os dados originais
df["Dia_Previsto"] = model.predict(X)

# Reverter a codificação para obter os nomes originais
df["Dia_Previsto"] = encoders["Dia da Semana"].inverse_transform(df["Dia_Previsto"])
df["Disciplina"] = encoders["Disciplina"].inverse_transform(df["Disciplina"])
df["Professor"] = encoders["Professor"].inverse_transform(df["Professor"])
df["Semestre"] = encoders["Semestre"].inverse_transform(df["Semestre"])

grade_aulas = df[["Semestre", "Disciplina", "Professor", "Dia_Previsto"]].sort_values(
    by=["Semestre", "Dia_Previsto"])

grade_aulas.head(60)

In [ ]:
dias_ordenados = ["Segunda-Feira", "Terça-Feira", "Quarta-Feira", "Quinta-Feira", "Sexta-Feira", "Sábado"]

semestres = grade_aulas["Semestre"].unique()

# Criar um dicionário para armazenar os DataFrames por semestre
dfs_semestres = {semestre: grade_aulas[grade_aulas["Semestre"] == semestre] for semestre in semestres}

dfs_transpostos = {}

for semestre, grade_aulas in dfs_semestres.items():
    tabela_transposta = grade_aulas.groupby("Dia_Previsto").agg(
        {'Disciplina': lambda x: list(set(x)), 'Professor': lambda x: list(set(x))}
    )
    
    # Transpor e ordenar as colunas (dias da semana)
    tabela_transposta = tabela_transposta.T
    
    # Reindexar as colunas para dias da semana
    tabela_transposta = tabela_transposta.reindex(columns=dias_ordenados, fill_value="")

    dfs_transpostos[semestre] = tabela_transposta


In [ ]:
# Salvar as tabelas em um único arquivo Excel
output_file = "tabela_aulas_organizada.xlsx"

with pd.ExcelWriter(output_file) as writer:
    for semestre, df_transposto in dfs_transpostos.items():
        df_transposto.to_excel(writer, sheet_name=f"Semestre_{semestre}")

print(f"Tabelas transpostas salvas em: {output_file}")
